In [1]:
##implement train/validation/test dataset for elaposolite formation energy

#load necessary package
#load elaposolite formation energy file


In [2]:
#load necessary package
import tensorflow as tf
import numpy as np
import pandas as pd
from pymatgen import Element,Composition

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
#load elaposolite formation energy file
elap_formation_energy_path="../data_set/evaluation_dataset/FE_elapsolite.txt"
data = pd.read_csv(elap_formation_energy_path,index_col=0)

In [4]:
data.head()

,compound,formation_energy
0,Ca1O1F2Cs6,-7.110018
1,Mg1S1Al2In6,4.713586
2,Cs1Te1S2Si6,18.374580
3,Be1Si1Tl2H6,4.612162
4,Mg1O1F2Ca6,-7.145500


In [5]:
data["compound"][0]

'Ca1O1F2Cs6'

In [6]:
##do max-min normalization for vector matrix?? if do ,then would be 0-1？feel bad
#can do like x_=(x-(max+min)/2)/(max-min)/2 to put x_ to [-1,1]
test_element_vector_path_1="../data_set/element_vector_generation/PCA/PCA_vector.csv"
test_element_vector_path_2="../data_set/element_vector_generation/laplace_eigenmap/laplace_eigenmap_20.txt"
test_output_path="../data_set/test_output/"

PCA_vector_raw=np.loadtxt(test_element_vector_path_1,delimiter=',')
LPE_vector_raw=np.loadtxt(test_element_vector_path_2)
#build input matrix(5628*80) 


In [7]:
#do normalization x_=(x-(max+min)/2)/(max-min) to put x_ to [-0.5,0.5]
#normalization for PCA_vector
PCA_vector_20_raw=PCA_vector_raw[:,:20]
PCA_MAX=np.max(PCA_vector_20_raw)
PCA_MIN=np.min(PCA_vector_20_raw)

PCA_NOM=(PCA_vector_20_raw-(PCA_MAX+PCA_MIN)/2)/(PCA_MAX-PCA_MIN)


In [8]:
#normalization for LPE_vector
LPE_MAX=np.max(LPE_vector_raw)
LPE_MIN=np.min(LPE_vector_raw)

LPE_NOM=(LPE_vector_raw-(LPE_MAX+LPE_MIN)/2)/(LPE_MAX-LPE_MIN)


In [9]:
#build the 5628*80 matrix
input_file_name_1="PCA_input_test.txt"
input_file_name_2="LPE_input_test.txt"

input_file_path_1=test_output_path+input_file_name_1
input_file_path_2=test_output_path+input_file_name_2

In [10]:
#load main_elements_list
element_list_main_path="../data_set/element_list/elements_list_main.txt"
#element_list_main_path_n="../data_set/element_list/elements_list_main_number.txt"
with open(element_list_main_path, "r") as text_file:
    a=text_file.read()
main_group_list=a.split()
#main_group_n=np.loadtxt(element_list_main_path_n,dtype=int)

In [11]:
#load elements_list
element_list_path="../data_set/element_list/elements.txt"
with open(element_list_path, "r") as text_file:
    a=text_file.read()
element_list=a.split()
#main_group_n=np.loadtxt(element_list_main_path_n,dtype=int)

In [12]:
type(element_list[0])

str

In [13]:
#ele_vector_length=20
#compound_list_length=5628
vector_input_LPE=np.zeros((len(data),LPE_NOM.shape[1]*len(Composition(data["compound"][0]))))
for i in range(len(data["compound"])):
    compound=Composition(data["compound"][i])
    #build row i [i,0:80]
    ele_index=0
    for ele in compound:
        for target_ele_index in range(len(element_list)):
            if str(ele)==element_list[target_ele_index]:
                vector_input_LPE[i,ele_index*20:(ele_index+1)*20]=LPE_NOM[target_ele_index,:]
                ele_index+=1

        #find the corresponding element

In [14]:
#ele_vector_length=20
#compound_list_length=5628
vector_input_PCA=np.zeros((len(data),LPE_NOM.shape[1]*len(Composition(data["compound"][0]))))
for i in range(len(data["compound"])):
    compound=Composition(data["compound"][i])
    #build row i [i,0:80]
    ele_index=0
    for ele in compound:
        for target_ele_index in range(len(element_list)):
            if str(ele)==element_list[target_ele_index]:
                vector_input_PCA[i,ele_index*20:(ele_index+1)*20]=PCA_NOM[target_ele_index,:]
                ele_index+=1

        #find the corresponding element

In [15]:
#save dataset to target folder/file

In [16]:
input_file_path_1

'../data_set/test_output/PCA_input_test.txt'

In [17]:
np.savetxt(input_file_path_1,vector_input_PCA)

In [18]:
input_file_path_2

'../data_set/test_output/LPE_input_test.txt'

In [19]:
np.savetxt(input_file_path_2,vector_input_LPE)

In [20]:
#input compound vector(5628*80)
#vector_input_PCA
#vector_input_LPE

#input compound formation energy(5628)
#data["formation_energy"]

In [21]:
data.head()

,compound,formation_energy
0,Ca1O1F2Cs6,-7.110018
1,Mg1S1Al2In6,4.713586
2,Cs1Te1S2Si6,18.374580
3,Be1Si1Tl2H6,4.612162
4,Mg1O1F2Ca6,-7.145500


In [22]:


##hold-out 10 to (80%,10%,10% for training/validation/test set)


In [23]:
vector_input_PCA.shape

(5628, 80)

In [24]:
PCA_NOM.shape

(80, 20)

In [25]:
#split the dataset into old-out 10 to (80%,10%,10% for training/validation/test set)
#hold number 10 is not implemented yet
X=vector_input_PCA
y=data["formation_energy"]

from sklearn.model_selection import train_test_split
seed=6

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=seed)

In [26]:
X_train.shape

(4502, 80)

In [27]:
X_val.shape

(563, 80)

In [28]:
X_test.shape

(563, 80)

In [29]:
#traing dataset: X_train/y_train
#validation dataset: X_val/y_val
#test dataset: X_test/y_test

In [36]:
#early_stop in the call back of earlystop
#build the model,train and test it.
#test dataset of PCA result
"""
keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')
"""
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
import tensorflow as tf


callback=EarlyStopping( monitor='val_loss',
                        min_delta=0,
                        patience=100,
                        verbose=0, mode='auto')

model = Sequential()
model.add(Dense(10, input_dim=80))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1))
          
model.compile(optimizer=Adam(lr=0.01,decay=1e-4),
              loss='mae',
              metrics=['mae'])

In [37]:
history=model.fit(X_train, y_train, validation_data=[X_val,y_val],epochs=1000,batch_size=32,callbacks=[callback])

Train on 4502 samples, validate on 563 samples
Epoch 1/1000
4502/4502 [==============================] - 1s 258us/step - loss: 4.6841 - mean_absolute_error: 4.6841 - val_loss: 3.1336 - val_mean_absolute_error: 3.1336
Epoch 2/1000
4502/4502 [==============================] - 0s 70us/step - loss: 3.2450 - mean_absolute_error: 3.2450 - val_loss: 2.5007 - val_mean_absolute_error: 2.5007
Epoch 3/1000
4502/4502 [==============================] - 0s 71us/step - loss: 2.7528 - mean_absolute_error: 2.7528 - val_loss: 2.3446 - val_mean_absolute_error: 2.3446
Epoch 4/1000
4502/4502 [==============================] - 0s 71us/step - loss: 2.6442 - mean_absolute_error: 2.6442 - val_loss: 2.5287 - val_mean_absolute_error: 2.5287
Epoch 5/1000
4502/4502 [==============================] - 0s 70us/step - loss: 2.5283 - mean_absolute_error: 2.5283 - val_loss: 2.1819 - val_mean_absolute_error: 2.1819
Epoch 6/1000
4502/4502 [==============================] - 0s 70us/step - loss: 2.4511 - mean_absolute_error

Epoch 49/1000
4502/4502 [==============================] - 0s 87us/step - loss: 2.0798 - mean_absolute_error: 2.0798 - val_loss: 1.6181 - val_mean_absolute_error: 1.6181
Epoch 50/1000
4502/4502 [==============================] - 0s 66us/step - loss: 2.1547 - mean_absolute_error: 2.1547 - val_loss: 1.6812 - val_mean_absolute_error: 1.6812
Epoch 51/1000
4502/4502 [==============================] - 0s 72us/step - loss: 2.0164 - mean_absolute_error: 2.0164 - val_loss: 1.6536 - val_mean_absolute_error: 1.6536
Epoch 52/1000
4502/4502 [==============================] - 0s 74us/step - loss: 2.1299 - mean_absolute_error: 2.1299 - val_loss: 1.7446 - val_mean_absolute_error: 1.7446
Epoch 53/1000
4502/4502 [==============================] - 0s 69us/step - loss: 2.1083 - mean_absolute_error: 2.1083 - val_loss: 1.6918 - val_mean_absolute_error: 1.6918
Epoch 54/1000
4502/4502 [==============================] - 0s 66us/step - loss: 2.0630 - mean_absolute_error: 2.0630 - val_loss: 1.6006 - val_mean_abs

4502/4502 [==============================] - 0s 98us/step - loss: 2.0061 - mean_absolute_error: 2.0061 - val_loss: 1.5132 - val_mean_absolute_error: 1.5132
Epoch 98/1000
4502/4502 [==============================] - 0s 93us/step - loss: 2.0103 - mean_absolute_error: 2.0103 - val_loss: 1.5256 - val_mean_absolute_error: 1.5256
Epoch 99/1000
4502/4502 [==============================] - 1s 131us/step - loss: 1.9631 - mean_absolute_error: 1.9631 - val_loss: 1.5620 - val_mean_absolute_error: 1.5620
Epoch 100/1000
4502/4502 [==============================] - 0s 95us/step - loss: 2.0373 - mean_absolute_error: 2.0373 - val_loss: 1.5069 - val_mean_absolute_error: 1.5069
Epoch 101/1000
4502/4502 [==============================] - 1s 121us/step - loss: 1.9947 - mean_absolute_error: 1.9947 - val_loss: 1.5404 - val_mean_absolute_error: 1.5404
Epoch 102/1000
4502/4502 [==============================] - 1s 112us/step - loss: 2.2008 - mean_absolute_error: 2.2008 - val_loss: 1.5307 - val_mean_absolute_er

Epoch 145/1000
4502/4502 [==============================] - 0s 65us/step - loss: 1.8870 - mean_absolute_error: 1.8870 - val_loss: 1.5003 - val_mean_absolute_error: 1.5003
Epoch 146/1000
4502/4502 [==============================] - 0s 65us/step - loss: 1.9811 - mean_absolute_error: 1.9811 - val_loss: 1.4948 - val_mean_absolute_error: 1.4948
Epoch 147/1000
4502/4502 [==============================] - 0s 63us/step - loss: 2.0339 - mean_absolute_error: 2.0339 - val_loss: 1.5095 - val_mean_absolute_error: 1.5095
Epoch 148/1000
4502/4502 [==============================] - 0s 61us/step - loss: 1.9554 - mean_absolute_error: 1.9554 - val_loss: 1.4895 - val_mean_absolute_error: 1.4895
Epoch 149/1000
4502/4502 [==============================] - 0s 61us/step - loss: 2.0463 - mean_absolute_error: 2.0463 - val_loss: 1.5023 - val_mean_absolute_error: 1.5023
Epoch 150/1000
4502/4502 [==============================] - 0s 63us/step - loss: 1.9554 - mean_absolute_error: 1.9554 - val_loss: 1.5039 - val_me

Epoch 193/1000
4502/4502 [==============================] - 0s 91us/step - loss: 1.9885 - mean_absolute_error: 1.9885 - val_loss: 1.4710 - val_mean_absolute_error: 1.4710
Epoch 194/1000
4502/4502 [==============================] - 0s 64us/step - loss: 2.0037 - mean_absolute_error: 2.0037 - val_loss: 1.4991 - val_mean_absolute_error: 1.4991
Epoch 195/1000
4502/4502 [==============================] - 0s 75us/step - loss: 2.0142 - mean_absolute_error: 2.0142 - val_loss: 1.5226 - val_mean_absolute_error: 1.5226
Epoch 196/1000
4502/4502 [==============================] - 0s 82us/step - loss: 1.9105 - mean_absolute_error: 1.9105 - val_loss: 1.5123 - val_mean_absolute_error: 1.5123
Epoch 197/1000
4502/4502 [==============================] - 0s 95us/step - loss: 1.9671 - mean_absolute_error: 1.9671 - val_loss: 1.4707 - val_mean_absolute_error: 1.4707
Epoch 198/1000
4502/4502 [==============================] - 0s 61us/step - loss: 2.0879 - mean_absolute_error: 2.0879 - val_loss: 1.4685 - val_me

Epoch 241/1000
4502/4502 [==============================] - 0s 61us/step - loss: 1.9802 - mean_absolute_error: 1.9802 - val_loss: 1.4711 - val_mean_absolute_error: 1.4711
Epoch 242/1000
4502/4502 [==============================] - 0s 61us/step - loss: 1.9708 - mean_absolute_error: 1.9708 - val_loss: 1.4679 - val_mean_absolute_error: 1.4679
Epoch 243/1000
4502/4502 [==============================] - 0s 61us/step - loss: 1.9855 - mean_absolute_error: 1.9855 - val_loss: 1.4881 - val_mean_absolute_error: 1.4881
Epoch 244/1000
4502/4502 [==============================] - 0s 61us/step - loss: 2.0255 - mean_absolute_error: 2.0255 - val_loss: 1.4770 - val_mean_absolute_error: 1.4770
Epoch 245/1000
4502/4502 [==============================] - 0s 61us/step - loss: 2.0018 - mean_absolute_error: 2.0018 - val_loss: 1.4627 - val_mean_absolute_error: 1.4627
Epoch 246/1000
4502/4502 [==============================] - 0s 60us/step - loss: 1.9240 - mean_absolute_error: 1.9240 - val_loss: 1.4741 - val_me

In [38]:
score = model.evaluate(X_test, y_test)

563/563 [==============================] - 0s 33us/step


In [39]:
score

[1.4365662658193394, 1.4365662658193394]

In [40]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 10)                810       
_________________________________________________________________
batch_normalization_3 (Batch (None, 10)                40        
_________________________________________________________________
activation_3 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 861
Trainable params: 841
Non-trainable params: 20
_________________________________________________________________


In [ ]:
#early_stop in the call back of earlystop
#build the model,train and test it.
#test dataset of PCA result
"""
keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')
"""
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
import tensorflow as tf


callback=EarlyStopping( monitor='val_loss',
                        min_delta=0,
                        patience=50,
                        verbose=0, mode='auto')

model = Sequential()
model.add(Dense(10, input_dim=80))
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dense(1))
          
model.compile(optimizer=Adam(lr=0.01,decay=1e-4),
              loss='mae',
              metrics=['mae'])

In [ ]:
X=vector_input_LPE
y=data["formation_energy"]

from sklearn.model_selection import train_test_split
seed=7

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=seed)

In [ ]:
history=model.fit(X_train, y_train, validation_data=[X_val,y_val],epochs=1000,batch_size=32)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
##comparison how effecient of our model
#do a distribution hestogram of our elements vector
#random a elements vector in that distribution
#do the same training for all the data

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
type(y)

In [ ]:
a=np.histogram(y,bins=20)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(y, bins='auto')
plt.title("Histogram with 'auto' bins")
plt.show()

In [ ]:
np.mean(np.dot(y,y))

In [ ]:
y2=np.square(y)

In [ ]:
np.mean(y2)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(X, bins='auto')
plt.title("Histogram with 'auto' bins")
plt.show()

In [ ]:
np.max(X)